In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.utils import convert_action

website to find .csv:

In [ ]:
df = pd.read_csv('./data/IBM.csv')
#df['Date'] = pd.to_datetime(df.Date, format="%d/%m/%Y")
#df = df.sort_values('Date')
# df.to_csv('./data/AAPL.csv')
df[['Date','Open','High','Low','Close','Volume']].head()

In [ ]:
df.plot(x="Date", y="Close")

In [ ]:
df.head()

## Let the DQN trade

In [ ]:
from networks.DQN import MyModel
from env.StockTradingEnv import StockTradingEnv
import tensorflow as tf

from tensorflow.keras.models import load_model

### Parameters of the DQN model (taken from main_with_dqn)

In [ ]:
env = StockTradingEnv(df)

num_actions = 2
hidden_units = [64,128,256,256,128,64]
input_shape = env.observation_space.sample().shape
in_log = True ### for the environment to use logs and no divisions by max value

path_weights_a = "save_models/DQN_200" # last number is number of episodes trained
path_weights_b = "save_models/DQN_400"
path_weights_c = "save_models/DQN_1000"

### Build the Agent

In [ ]:
class TradingAgent():
    """A class to simplify the trading decisions"""
    def __init__(self, model, path_weights, name):
        self.model = model
        self.model.load_weights(path_weights)
        self.name = name # name of the agent
    
    def get_action(self, state): #meant for only a single state for input
        prediction = self.model.predict(state)
        return np.argmax(prediction[0])

In [ ]:
trade_model_a = MyModel(input_shape, hidden_units, num_actions)
trade_model_b = MyModel(input_shape, hidden_units, num_actions)
trade_model_c = MyModel(input_shape, hidden_units, num_actions)

# trade_model.load_weights(path_weights)
Agent_a = TradingAgent(trade_model_a, path_weights_a, "agent_a")
Agent_b = TradingAgent(trade_model_b, path_weights_b, "agent_b")
Agent_c = TradingAgent(trade_model_c, path_weights_c, "agent_c")

agents = [Agent_a, Agent_b, Agent_c]

### Run over the data frame

In [ ]:
df_results = pd.DataFrame(columns=['Date','Index', 'agent_a', 'agent_b', 'agent_c'], index=df.index)
df_results['Date'] = df['Date']

# Compute the Index value
for i in range(len(df.loc[:, 'Open'].values) - 6):
    df_results.loc[i+5, "Index"] = df.loc[i+5, "Close"] * env.initial_account_balance / df.loc[5, "Close"] # i + 5 because of the format of the data
    
# Compute the Agents' net_worth
for Agent in agents:
    env = StockTradingEnv(df, in_log=in_log)
    input_shape = env.observation_space.sample().shape
    observation = env.reset_to_day_one().reshape(1,input_shape[0],input_shape[1]).astype('float32') # necessary to reshape each observation
    
    for i in range(len(df.loc[:, 'Open'].values) - 6):
        action = Agent.get_action(observation.reshape(1,input_shape[0],input_shape[1]).astype('float32'))
        action = convert_action(action, binary_action=True)
        observation, reward, done, _ = env.step(action)

        df_results.loc[i+5, Agent.name] = env.net_worth

        print("\r {}, day {} out of {}".format(Agent.name, i, len(df.loc[:, 'Open'].values) - 7), end = "")
    print("")
    
    print("last net_worth: {}".format(env.net_worth))

### Visualize the results

#### On training set

#### On validation set

In [ ]:
fig = df_results.plot(x="Date", y=["Index", "agent_a", "agent_b", "agent_c"]).get_figure()
fig.savefig("imgs/DQN_mlp.png")